In [1]:
import pandas as pd

In [ ]:
import re

def parse_lean_code(formatted_code):
    match = re.search(r'```lean(.*?)```', formatted_code, re.DOTALL)
    if match:
        lean_code = match.group(1).strip()  # Extract the Lean code and strip any surrounding whitespace
        return lean_code
    else:
        raise ValueError("No valid Lean 4 code block found.")
   
def get_prompt(statement, solution_step, lean_codes):
    if lean_codes=="":
        return f"""
        You are a Lean 4 expert and your task is to formalize mathematical proofs step by step. You will be provided with a problem statement, conditions, and a natural language solution for the first step. Your job is to formalize this first solution step into Lean 4 code, using appropriate axioms, theorems, and Lean tactics.
        
        Instructions:
        1. **Analyze the problem statement** and extract the relevant assumptions.
        2. **Translate the first step** of the solution into formal Lean 4 code.
        3. Ensure that the proof compiles correctly in Lean 4.
        4. Output the Lean 4 code enclosed in the following format for easy parsing:
        ```lean
        <Lean 4 code>
        ```
        
        Input:
        - {statement}
        - Solution: {solution_step}
        """
    else:
        return f"""
        You are a Lean 4 expert and your task is to formalize mathematical proofs step by step. You will be provided with a problem statement, previous formalized proofs, and a natural language solution for the current step. Your job is to reuse the previous Lean proof by treating it as a hypothesis, and formalize the current step into Lean 4 code, using appropriate axioms, theorems, and Lean tactics.
        
        Instructions:
        1. **Reuse the previous Lean proof** as a hypothesis in the new proof. Treat the previous theorem as already verified and use it in the current step.
        2. **Translate the new solution step** into formal Lean 4 code, ensuring it builds on the hypothesis from the previous proof.
        3. Ensure that the proof compiles correctly in Lean 4.
        4. Output the Lean 4 code enclosed in the following format for easy parsing:
        ```lean
        <Lean 4 code>
        ```
        
        ### Specific Instructions for Reusing the Previous Proof:
        - Include the previous proof as a hypothesis. For example, if the previous theorem was `f_of_one_is_one`, reuse it as a hypothesis like:
        ```lean
        have h_f1 := f_of_one_is_one f h1 h2 h3
        ```
                
        Input:
        - {statement}
        - Next Solution Step: {solution_step}
        - Previous Formal Steps:\n{lean_codes}
        """

In [2]:
problem_id = 0

In [14]:
df =pd.read_pickle(f"../data/problem_{problem_id}.pkl")
lean_codes = ""
for i in range(1, len(df) + 1):
    statement = df[df["num_steps"] == i].statement.iat[0]
    solution_step = df[df["num_steps"] == i].solution_steps.iat[0][-1]
    retry = 0
    lean_code = ""
    valid = False
    while not valid and retry < 10:
        try:
            prompt = get_prompt(statement, solution_step, lean_codes)
            output = get_response(prompt)
            lean_code = parse_lean_code(output)
            lean_codes += "\n\n" + lean_code
            write_lean_file(lean_codes, problem_id, i)
            valid = verify_lean_code(problem_id, i)
        except Exception as e:
            print(f"error: {e}")
        retry += 1
    if not valid:
        break

2
3
4
5
6
7
